<a href="https://colab.research.google.com/github/ARJUN108-verma/Elite_Tech_internship/blob/main/NEURAL_STYLE_TRANSFER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NEURAL STYLE TRANSFER

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import time
import IPython.display as display

In [2]:
# Configuration
content_path = 'content.jpg'  # Replace with your content image path
style_path = 'style.jpg'     # Replace with your style image path
output_prefix = 'stylized_'
iterations = 1000
content_weight = 1e4
style_weight = 1e-2
total_variation_weight = 30

In [3]:
# Image dimensions (adjust based on your GPU memory)
img_width, img_height = 512, 512

In [4]:
def preprocess_image(image_path):
    """Load and preprocess an image for VGG19"""
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

In [5]:
def deprocess_image(x):
    """Convert a tensor back to an image"""
    x = x.reshape((img_height, img_width, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [6]:
def content_loss(base, combination):
    """Content loss between base and generated images"""
    return K.sum(K.square(combination - base))


In [7]:
def gram_matrix(x):
    """Compute Gram matrix for style representation"""
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [8]:
def style_loss(style, combination):
    """Style loss between style and generated images"""
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [9]:
def total_variation_loss(x):
    """Total variation loss for spatial smoothness"""
    a = K.square(
        x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(
        x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [10]:
# Build VGG19 model with pretrained ImageNet weights
def get_model():
    """Build VGG19 model returning intermediate layer outputs"""
    model = vgg19.VGG19(weights='imagenet', include_top=False)
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
    return tf.keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [11]:
# Get content and style features
def get_features(model, content_image, style_image):
    """Extract content and style features from images"""
    # Combine both images in single batch
    batch = tf.concat([content_image, style_image], axis=0)
    # Get features from all layers
    features = model(batch)

    # Separate content and style features
    content_features = {
        layer_name: features[layer_name][0]
        for layer_name in content_layers}
    style_features = {
        layer_name: features[layer_name][1]
        for layer_name in style_layers}

    return content_features, style_features

In [12]:
# Layers for content and style representation
content_layers = ['block5_conv2']
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1'
]

In [16]:
# Compute loss between generated image and targets
def compute_loss(model, generated_image, content_features, style_features):
    """Compute total loss (content + style + variation)"""
    # Combine generated image with original images in single batch
    batch = tf.concat([generated_image, content_image, style_image], axis=0)
    features = model(batch)

    # Initialize loss
    loss = 0

    # Content loss
    layer_features = features[content_layers[0]]
    generated_features = layer_features[0]
    loss += content_weight * content_loss(content_features[content_layers[0]], generated_features)

    # Style loss
    for layer_name in style_layers:
        generated_features = features[layer_name][0]
        loss += (style_weight / len(style_layers)) * style_loss(
            style_features[layer_name], generated_features)

    # Total variation loss
    loss += total_variation_weight * total_variation_loss(generated_image)
    return loss

In [20]:
@tf.function
def train_step(generated_image, model, content_features, style_features):
    """Single training step with gradient calculation"""
    with tf.GradientTape() as tape:
        loss = compute_loss(model, generated_image, content_features, style_features)

    grad = tape.gradient(loss, generated_image)
    optimizer.apply_gradients([(grad, generated_image)])
    generated_image.assign(tf.clip_by_value(generated_image, clip_value_min=-1.5, clip_value_max=1.5))
    return loss

In [18]:
def style_transfer(content_path, style_path, iterations=1000):
    """Perform neural style transfer"""
    # Preprocess images
    global content_image, style_image
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Initialize generated image with content image
    generated_image = tf.Variable(content_image)

    # Create model and extract features
    model = get_model()
    content_features, style_features = get_features(model, content_image, style_image)

    # Optimizer
    global optimizer
    optimizer = tf.optimizers.Adam(learning_rate=5.0, beta_1=0.99, epsilon=1e-1)

    # Display intermediate results
    display_interval = iterations // 10
    if display_interval == 0:
        display_interval = 100

    print("Starting style transfer...")
    start_time = time.time()

    for i in range(iterations):
        loss = train_step(generated_image, model, content_features, style_features)

        if i % display_interval == 0:
            print(f"Iteration {i}: loss={loss:.2f}")
            img = deprocess_image(generated_image.numpy())
            plt.imshow(img)
            plt.axis('off')
            plt.show()

    print(f"Finished in {time.time() - start_time:.2f} seconds")
    return generated_image

In [25]:
import os
print(os.getcwd())
content_path = '/full/path/to/content.jpg'
from google.colab import files
uploaded = files.upload()
import os
print(os.listdir())



/content


Saving my photo.jpg to my photo.jpg
['.config', 'my photo-Photoroom.png', 'my photo.jpg', 'sample_data']


In [26]:
from google.colab import files
uploaded = files.upload()
content_path = 'your_uploaded_content.jpg'
style_path = 'your_uploaded_style.jpg'


Saving my photo.jpg to my photo (1).jpg
